In [22]:
import os
import cv2
import numpy as np
import threading
import pywt
import scipy.fftpack
import queue
from queue import Queue
channel = 0 #0:blue/y,1:green/cg,2:red/co


images_DIR="embedded\\"
def get_image_names(folder_path):
    image_names = []
    for filename in os.listdir(folder_path):
        image_names.append(filename)
    return image_names
image_names = get_image_names(images_DIR)

In [23]:
def extract_middle_portion(A, m, n, x, y):
 

  if x > m or y > n:
    raise ValueError("x and y must be less than or equal to the corresponding dimensions of the matrix.")

  # Calculate the starting indices for the middle portion.
  start_row = (m - x) // 2
  start_col = (n - y) // 2

  # Extract the middle portion.
  return A[start_row:start_row+x, start_col:start_col+y]

In [24]:


def deconcatenate_2d_to_color_image(matrix, dims):

  # Reshape the matrix back into a 3D matrix.
  image = np.reshape(matrix, dims)

  # Return the deconcatenated image.
  return image



In [25]:
def rgb_to_ycbcr_lossless(img_bgr):
  matrix = np.array([[0.299, 0.587, 0.114],[-0.168736, -0.331264, 0.5],[0.5, -0.418688, -0.081312]], dtype=np.float64)
  img_ycrcb = np.dot(img_bgr, matrix.T)
  if (channel == 0):
      return img_ycrcb
 
  return img_bgr



In [26]:


def row_even_odd_split(array):
    
  even_array = array[::2, :]
  odd_array = array[1::2, :]
  return even_array, odd_array

def column_even_odd_split(array):

  even_array = array[:, ::2]
  odd_array = array[:, 1::2]
  return even_array, odd_array



def even_odd_split(matrix):
    flag = False
    last_elements = None
    if matrix.shape[0]>=matrix.shape[1]:#if row length > column length a or square matrix
        flag = True
        if matrix.shape[0]%2 != 0:
            last_elements = matrix[-1, :]
            matrix = matrix[:-1, :]
        even, odd = row_even_odd_split(matrix)              
    else:
        print(matrix.shape) #if row length < column length a
        if matrix.shape[1]%2 != 0:
            last_elements = matrix[:, -1]
            matrix = matrix[:, :-1]
        even, odd = column_even_odd_split(matrix)
        print(matrix.shape)

    return even, odd, flag, last_elements



In [27]:


def recover_watermark(A,shape,alpha=0.1):#recover_watermark(A, rowW, colW, alpha=0.1):
    alpha = np.float64(alpha)
    two = np.float64(2.)
    
    Wrow = shape[0]
    Wcol = shape[1] * shape[2]
    
    evenw, oddw, flag, last= even_odd_split(A)
    
 

    roweven, coleven = evenw.shape
    
    
    ZZeven = extract_middle_portion(evenw.copy(), roweven, coleven, Wrow, Wcol)
    ZZodd = extract_middle_portion(oddw.copy(), roweven, coleven,Wrow, Wcol)


    

    # Initialize matrix W with zeros
    flattenW = np.zeros((Wrow,Wcol), dtype=np.float64)

    # Iterate through even indices of A and recover B using the specified equation

    flattenW = ((ZZeven - ZZodd)/two)
    flattenW= flattenW/alpha
    
    
    return flattenW


In [28]:
def process4extract_image(image_name, MDIR, XDIR, shape=(32,32,3)):
  wimage_path = MDIR+image_name
  image1 = cv2.imread(wimage_path)
  image = rgb_to_ycbcr_lossless(image1)
  bwm =image[:, :, channel]

  if bwm.shape[0] % 4 != 0:
        rowLen=bwm.shape[0] % 4
        last_row = bwm[-rowLen:,:]
        bwm= bwm[:-rowLen,:]
  
  if bwm.shape[1] % 4 != 0:
        colLen=bwm.shape[1] % 4
        last_column = bwm[:,-colLen:]
        bwm= bwm[:,:-colLen]

  #bwm_LL, (bwm_LH, bwm_HL, bwm_HH) = pywt.dwt2(bwm, 'db1')
  values = pywt.wavedec2(bwm, 'coif3', mode='periodization', level=2)
  (b_LH, b_HL, b_HH) = values[1]


    
  water = recover_watermark(b_HH, shape)
  watermark_ext = deconcatenate_2d_to_color_image(water,shape)

  if not image_name.endswith(".png"):
    image_name = image_name.split(".")[0] + ".png"
  extracted_water_path = XDIR+"extracted_from_"+image_name
  cv2.imwrite(extracted_water_path, watermark_ext)



In [29]:
def Xtract(MDIR,XDIR):
    os.makedirs(XDIR, exist_ok=True)
    embedd_image_names = get_image_names(MDIR)
    threads = []
    for imgname in embedd_image_names:
        thread = threading.Thread(target=process4extract_image, args=(imgname,MDIR,XDIR))
        threads.append(thread)


    for thread in threads:
        thread.start()


    for thread in threads:
        thread.join()

In [30]:
def multithread(cell):
    threads = [] 
    for imgname in image_names:
        thread = threading.Thread(target=cell, args=(imgname,))
        threads.append(thread) # add the thread to the list
        thread.start() # start the thread

    for thread in threads:
        thread.join()

In [31]:
%%time
def downscale(convertedDIR):
    def convertion(image_name):
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        scale_factor = 0.75  # Adjust the scaling factor as needed
        h, w, _ = image.shape
        attacked_image = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))
        restored_image = cv2.resize(attacked_image, (w, h))
        cv2.imwrite(convertedDIR+image_name, restored_image)


    multithread(convertion)
convertedDIR="attack\\images\\downscale\\"
os.makedirs(convertedDIR, exist_ok=True)
downscale(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\downscale\\"
Xtract(MDIR,XDIR)

CPU times: total: 297 ms
Wall time: 277 ms


In [32]:
%%time
def upscale(convertedDIR):
    def convertion(image_name):
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        scale_factor = 1.5  # Adjust the scaling factor as needed
        h, w, _ = image.shape
        attacked_image = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))
        restored_image = cv2.resize(attacked_image, (w, h))
        cv2.imwrite(convertedDIR+image_name, restored_image)


    multithread(convertion)
convertedDIR="attack\\images\\upscale\\"
os.makedirs(convertedDIR, exist_ok=True)
upscale(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\upscale\\"
Xtract(MDIR,XDIR)

CPU times: total: 93.8 ms
Wall time: 167 ms


In [33]:
%%time
def gaussBlurr(convertedDIR):
    def convertion(image_name):
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        kernel_size = (3, 3)  # Adjust the kernel size as needed
        blurred_image = cv2.GaussianBlur(image, kernel_size, 0)
        cv2.imwrite(convertedDIR+image_name, blurred_image)


    multithread(convertion)
convertedDIR="attack\\images\\gaussBlurr\\"
os.makedirs(convertedDIR, exist_ok=True)
gaussBlurr(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\gaussBlurr\\"
Xtract(MDIR,XDIR)

CPU times: total: 125 ms
Wall time: 164 ms


In [34]:
%%time
def rotation(convertedDIR):
    def convertion(image_name):
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        rotation_angle = 0.3  # Adjust the rotation angle as needed
        rows, cols, _ = image.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), rotation_angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

        cv2.imwrite(convertedDIR+image_name, rotated_image)

    multithread(convertion)

convertedDIR="attack\\images\\rotation\\"
os.makedirs(convertedDIR, exist_ok=True)
rotation(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\rotation\\"
Xtract(MDIR,XDIR)

CPU times: total: 31.2 ms
Wall time: 154 ms


In [35]:
%%time
def saltpepper(convertedDIR):
    def convertion(image_name):
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        noise_density = 0.001  # Adjust the noise density as needed

        # Create a copy of the watermarked image
        noisy_image = image.copy()

        # Generate random noise
        salt_and_pepper = np.random.rand(noisy_image.shape[0], noisy_image.shape[1])

        # Add salt noise
        noisy_image[salt_and_pepper < noise_density] = 255

        # Add pepper noise
        noisy_image[salt_and_pepper > 1 - noise_density] = 0

        cv2.imwrite(convertedDIR+image_name, noisy_image)


    multithread(convertion)
convertedDIR="attack\\images\\saltpepper\\"
os.makedirs(convertedDIR, exist_ok=True)
saltpepper(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\saltpepper\\"
Xtract(MDIR,XDIR)

CPU times: total: 93.8 ms
Wall time: 187 ms


In [36]:
%%time
def jpegcompression(convertedDIR):
    def convertion(image_name):
        
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image)


    multithread(convertion)
convertedDIR="attack\\images\\jpeg\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg\\"
Xtract(MDIR,XDIR)

CPU times: total: 62.5 ms
Wall time: 148 ms


In [37]:
%%time
def jpegcompression50(convertedDIR):
    def convertion(image_name):
        compression_quality = 50
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

    multithread(convertion)
convertedDIR="attack\\images\\jpeg50\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression50(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg50\\"
Xtract(MDIR,XDIR)

CPU times: total: 93.8 ms
Wall time: 140 ms


In [38]:
%%time
def jpegcompression30(convertedDIR):
    def convertion(image_name):
        compression_quality = 30
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

    multithread(convertion)
convertedDIR="attack\\images\\jpeg30\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression30(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg30\\"
Xtract(MDIR,XDIR)

CPU times: total: 141 ms
Wall time: 160 ms


In [39]:
%%time
def jpegcompression40(convertedDIR):
    def convertion(image_name):
        compression_quality = 40
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

    multithread(convertion)
convertedDIR="attack\\images\\jpeg40\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression40(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg40\\"
Xtract(MDIR,XDIR)

CPU times: total: 141 ms
Wall time: 164 ms


In [40]:
%%time
def jpegcompression60(convertedDIR):
    def convertion(image_name):
        compression_quality = 60
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

    multithread(convertion)
convertedDIR="attack\\images\\jpeg60\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression60(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg60\\"
Xtract(MDIR,XDIR)

CPU times: total: 78.1 ms
Wall time: 134 ms


In [41]:
%%time
def jpegcompression70(convertedDIR):
    def convertion(image_name):
        compression_quality = 70
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

    multithread(convertion)
convertedDIR="attack\\images\\jpeg70\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression70(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg70\\"
Xtract(MDIR,XDIR)

CPU times: total: 15.6 ms
Wall time: 128 ms


In [42]:
%%time
def jpegcompression80(convertedDIR):
    def convertion(image_name):
        compression_quality = 80
        image_path = images_DIR+image_name
        image = cv2.imread(image_path)
        
        cv2.imwrite(convertedDIR + image_name[:-4] + '.jpg', image, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

    multithread(convertion)
convertedDIR="attack\\images\\jpeg80\\"
os.makedirs(convertedDIR, exist_ok=True)
jpegcompression80(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\jpeg80\\"
Xtract(MDIR,XDIR)

CPU times: total: 31.2 ms
Wall time: 133 ms


In [43]:

#bright>1.0 dark 0<x<1.0 
def Bright15(convertedDIR):
  def conversion(image_name):
    factor = 1.5
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright15\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright15(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright15\\"
Xtract(MDIR,XDIR)

In [44]:
def Bright14(convertedDIR):
  def conversion(image_name):
    factor = 1.4
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright14\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright14(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright14\\"
Xtract(MDIR,XDIR)

In [45]:
def Bright13(convertedDIR):
  def conversion(image_name):
    factor = 1.3
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright13\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright13(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright13\\"
Xtract(MDIR,XDIR)

In [46]:
def Bright12(convertedDIR):
  def conversion(image_name):
    factor = 1.2
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright12\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright12(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright12\\"
Xtract(MDIR,XDIR)

In [47]:
def Bright11(convertedDIR):
  def conversion(image_name):
    factor = 1.1
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright11\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright11(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright11\\"
Xtract(MDIR,XDIR)

In [48]:
def Bright09(convertedDIR):
  def conversion(image_name):
    factor = 0.9
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright09\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright09(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright09\\"
Xtract(MDIR,XDIR)

In [49]:
def Bright08(convertedDIR):
  def conversion(image_name):
    factor = 0.8
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright08\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright08(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright08\\"
Xtract(MDIR,XDIR)

In [50]:
def Bright07(convertedDIR):
  def conversion(image_name):
    factor = 0.7
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright07\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright07(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright07\\"
Xtract(MDIR,XDIR)

In [51]:
def Bright06(convertedDIR):
  def conversion(image_name):
    factor = 0.6
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright06\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright06(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright06\\"
Xtract(MDIR,XDIR)

In [52]:
#bright>1.0 dark 0<x<1.0 
def Bright05(convertedDIR):
  def conversion(image_name):
    factor = 0.5
    image_path = images_DIR + image_name
    image = cv2.imread(image_path)

    # Adjust brightness using element-wise multiplication
    brighter_image = image * factor

    # Clip pixel values to prevent overflow/underflow (optional)
    #brighter_image = np.clip(brighter_image, 0, 255, out=brighter_image, casting="uint8")

    cv2.imwrite(convertedDIR + image_name, brighter_image)

  multithread(conversion)


convertedDIR="attack\\images\\bright05\\"
os.makedirs(convertedDIR, exist_ok=True)
Bright05(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\bright05\\"
Xtract(MDIR,XDIR)

In [53]:
def crop25UL(convertedDIR):
  def conversion(image_name):
    image_path = images_DIR+image_name
    image = cv2.imread(image_path)

    # Get the height and width of the image
    height, width, _ = image.shape

    # Calculate the area to be replaced (25% of the image)
    area_height = int(height * 0.50)
    area_width = int(width * 0.50)

    # Replace the upper left corner with black pixels
    image[0:area_height, 0:area_width] = (0, 0, 0)

    cv2.imwrite(convertedDIR+image_name, image)

  multithread(conversion)


convertedDIR="attack\\images\\crop25UL\\"
os.makedirs(convertedDIR, exist_ok=True)
crop25UL(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\crop25UL\\"
Xtract(MDIR,XDIR)

In [54]:
def crop25W(convertedDIR):
  def conversion(image_name):
    image_path = images_DIR+image_name
    image = cv2.imread(image_path)

    # Get the height and width of the image
    height, width, _ = image.shape

    # Replace the boundary (1 pixel wide) with black pixels
    boundary=50 #25%
    image[0:boundary, 0:width] = (0, 0, 0)  # top boundary
    image[height-boundary:height, 0:width] = (0, 0, 0)  # bottom boundary
    image[0:height, 0:boundary] = (0, 0, 0)  # left boundary
    image[0:height, width-boundary:width] = (0, 0, 0)  # right boundary

    cv2.imwrite(convertedDIR+image_name, image)


  multithread(conversion)


convertedDIR="attack\\images\\crop25W\\"
os.makedirs(convertedDIR, exist_ok=True)
crop25W(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\crop25W\\"
Xtract(MDIR,XDIR)

In [55]:
def noise_attack(convertedDIR):
    def conversion(image_name):
        image_path = images_DIR + image_name
        image = cv2.imread(image_path)

        # Add Gaussian noise
        noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
        noisy_image = cv2.add(image, noise)

        cv2.imwrite(convertedDIR + image_name, noisy_image)

    multithread(conversion)

convertedDIR = "attack\\images\\noise\\"
os.makedirs(convertedDIR, exist_ok=True)
noise_attack(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\noise\\"
Xtract(MDIR,XDIR)


In [56]:
def color_channel_attack(convertedDIR):
    def conversion(image_name):
        image_path = images_DIR + image_name
        image = cv2.imread(image_path)

        # Set one color channel (e.g., Red) to zero
        image[:, :, 2] = 0  # Set the Red channel to zero

        cv2.imwrite(convertedDIR + image_name, image)

    multithread(conversion)

convertedDIR = "attack\\images\\color_channel\\"
os.makedirs(convertedDIR, exist_ok=True)
color_channel_attack(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\color_channel\\"
Xtract(MDIR,XDIR)


In [57]:
def histogram_equalization_attack(convertedDIR):
    def conversion(image_name):
        image_path = images_DIR + image_name
        image = cv2.imread(image_path)

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Apply histogram equalization
        equalized = cv2.equalizeHist(gray)
        # Convert back to BGR
        equalized_image = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)

        cv2.imwrite(convertedDIR + image_name, equalized_image)

    multithread(conversion)

convertedDIR = "attack\\images\\hist_equal\\"
os.makedirs(convertedDIR, exist_ok=True)
histogram_equalization_attack(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\hist_equal\\"
Xtract(MDIR,XDIR)


In [58]:
def patch_removal_attack(convertedDIR, patch_size=0.2):
    def conversion(image_name):
        image_path = images_DIR + image_name
        image = cv2.imread(image_path)

        # Define patch dimensions
        height, width, _ = image.shape
        patch_h = int(height * patch_size)
        patch_w = int(width * patch_size)

        # Remove a central patch
        image[height//2-patch_h//2:height//2+patch_h//2, width//2-patch_w//2:width//2+patch_w//2] = (255, 255, 255)

        cv2.imwrite(convertedDIR + image_name, image)

    multithread(conversion)

convertedDIR = "attack\\images\\patch_removal\\"
os.makedirs(convertedDIR, exist_ok=True)
patch_removal_attack(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\patch_removal\\"
Xtract(MDIR,XDIR)



In [59]:
def frequency_filter_attack(convertedDIR):
    def conversion(image_name):
        image_path = images_DIR + image_name
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Perform FFT
        f = np.fft.fft2(image)
        fshift = np.fft.fftshift(f)

        # Apply a high-pass filter by zeroing out low-frequency components
        rows, cols = image.shape
        crow, ccol = rows//2, cols//2
        fshift[crow-30:crow+30, ccol-30:ccol+30] = 0

        # Perform inverse FFT
        ishift = np.fft.ifftshift(fshift)
        filtered_image = np.abs(np.fft.ifft2(ishift))

        cv2.imwrite(convertedDIR + image_name, filtered_image)

    multithread(conversion)

convertedDIR = "attack\\images\\frequency_filter\\"
os.makedirs(convertedDIR, exist_ok=True)
frequency_filter_attack(convertedDIR)

MDIR = convertedDIR
XDIR ="attack\\extracted1\\frequency_filter\\"
Xtract(MDIR,XDIR)
